In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
adm = pd.read_csv("../data/hos/admissions.csv",  header=0, index_col=None, parse_dates=['admittime'])


In [3]:
adm.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [4]:
pts = pd.read_csv('../data/hos/patients.csv', header=0, index_col=None, parse_dates=['dod']).fillna(0)

In [5]:
# Get the MAXIMUM possible year that the anchor_year corresponds to
pts['min_year_group'] = pts.anchor_year_group.str.slice(start=0, stop=4).astype(int)
pts['max_year_group'] = pts.anchor_year_group.str.slice(start=-4).astype(int)

# Youngest possible age occurs by finding the diff between the max year and 2008, then subtracting from anchor_age
# This ensures you don't accidentally include someone who might've been <18 in that anchor year range during the 2008-2019 window
pts['min_age'] = pts.anchor_age.values - (pts.max_year_group.values - 2008)
# pts = pts.loc[pts.min_age >= 18]

# gets a patient's specific anchor year that corresponds to the range 2008-2010
# This year is timedelta = 0 for the end dataset
pts['base_anchor_year'] = (pts.anchor_year - (pts.min_year_group - 2008))

In [6]:
pts.head() #(259289, 10)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,min_year_group,max_year_group,min_age,base_anchor_year
0,10000032,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
1,10000048,F,23,2126,2008 - 2010,0,2008,2010,21,2126
2,10000068,F,19,2160,2008 - 2010,0,2008,2010,17,2160
3,10000084,M,72,2160,2017 - 2019,2161-02-13 00:00:00,2017,2019,61,2151
4,10000102,F,27,2136,2008 - 2010,0,2008,2010,25,2136


In [7]:
print(pts.shape)
print('mean-age in the dataset:', round(pts['min_age'].mean(),2))
print('minimum-age in the dataset:', round(pts['min_age'].min(),2))
print('maximum-age in the dataset:', round(pts['min_age'].max(),2))
print('mean-age in the dataset:', round(pts['anchor_age'].mean(),2))
print('minimum-age in the dataset:', round(pts['anchor_age'].min(),2))
print('maximum-age in the dataset:', round(pts['anchor_age'].max(),2))
print('Gender: Female-', len(pts.loc[pts['gender']=='F']))
print('Gender: Male-', len(pts.loc[pts['gender']=='M']))

(299712, 10)
mean-age in the dataset: 42.58
minimum-age in the dataset: 7
maximum-age in the dataset: 89
mean-age in the dataset: 48.54
minimum-age in the dataset: 18
maximum-age in the dataset: 91
Gender: Female- 158553
Gender: Male- 141159


In [8]:
pts['min_age'].value_counts()

#get the 5 year bucket for plotting

18    6896
19    6451
17    6392
20    6069
16    5865
      ... 
82    1167
84     992
7      826
85     623
87     347
Name: min_age, Length: 82, dtype: int64

In [9]:
print(adm.columns.to_list())
print(pts.columns.to_list())

['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag']
['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod', 'min_year_group', 'max_year_group', 'min_age', 'base_anchor_year']


In [10]:
#merge the paitent and admissions

adm_pts = (adm.merge(pts, 
           how='inner', left_on=['subject_id'], right_on=['subject_id'])
)

In [11]:
#finding combination for patients and how many times they came to hospital

adm_pts.shape #(412850, 25)

adm_pts['subject_id'].nunique() #168473

adm_pts['hadm_id'].nunique() #412850

(adm_pts.groupby(adm_pts['subject_id'])['hadm_id']
 .nunique().to_frame().rename(columns={'hadm_id': 'hadm_id_counts'})
 .sort_values('hadm_id_counts', ascending=False))

,hadm_id_counts
subject_id,
15496609,238
15464144,185
10714009,163
16662316,142
15229574,130
...,...
12377951,1
15931810,1
15931659,1


In [12]:
adm_pts.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,min_year_group,max_year_group,min_age,base_anchor_year
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,...,0,F,19,2160,2008 - 2010,0,2008,2010,17,2160


In [13]:
pts_diag = pd.read_csv('../data/hos/diagnoses_icd.csv').reset_index(drop = True)

In [14]:
print(pts_diag.shape)
pts_diag.head(10)

(4756326, 5)


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
5,10000032,22595853,6,29680,9
6,10000032,22595853,7,30981,9
7,10000032,22595853,8,V1582,9
8,10000032,22841357,1,07071,9
9,10000032,22841357,2,78959,9


In [15]:
pts_diag['icd_version'].value_counts()

9     2766877
10    1989449
Name: icd_version, dtype: int64

In [16]:
#normalization of ic9 and icd10 codes.

In [17]:
df = pts_diag.head(40)

In [18]:
df.head(20)

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
5,10000032,22595853,6,29680,9
6,10000032,22595853,7,30981,9
7,10000032,22595853,8,V1582,9
8,10000032,22841357,1,07071,9
9,10000032,22841357,2,78959,9


In [19]:
df.pivot(index=['subject_id','hadm_id','icd_version'], columns='icd_code').reset_index()

subject_id   hadm_id icd_version seq_num                              \
icd_code                                    07044 07054 07070 07071 2761 2767   
0          10000032  22595853           9     NaN   NaN   4.0   NaN  NaN  NaN   
1          10000032  22841357           9     NaN   NaN   NaN   1.0  4.0  NaN   
2          10000032  25742920           9     NaN   1.0   NaN   NaN  6.0  5.0   
3          10000032  29079034           9     2.0   NaN   NaN   NaN  4.0  6.0   
4          10000068  25022803           9     NaN   NaN   NaN   NaN  NaN  NaN   

               ...                                                           
icd_code 2875  ...   496  5715 5723 78791 78959 7994  V08 V1582  V462 V4986  
0         NaN  ...   5.0   3.0  1.0   NaN   2.0  NaN  NaN   8.0   NaN   NaN  
1         3.0  ...   5.0   6.0  NaN   NaN   2.0  NaN  7.0   NaN   NaN   NaN  
2         NaN  ...   7.0   4.0  NaN  10.0   2.0  NaN  8.0   NaN   3.0   NaN  
3         NaN  ...  11.0  13.0  NaN   NaN   5.0  3.0  8.0   NaN  10.0   9.0  
4         NaN  ...   NaN   NaN  NaN   NaN   NaN  NaN  NaN   NaN   NaN   NaN  

[5 rows x 25 columns]

In [20]:
pts_diag.groupby(['subject_id', 'hadm_id', 'icd_version'],  as_index = False)['icd_code'].count().sort_values('icd_code', ascending =False)

#14975184	27635276 57

,subject_id,hadm_id,icd_version,icd_code
8676,10213059,29029082,10,39
162464,13791947,27054007,10,39
257798,15980060,26057495,10,39
327309,17595401,21613580,9,39
120225,12800814,29909244,10,39
...,...,...,...,...
303524,17044572,20747759,9,1
234389,15447207,29372164,9,1
420962,19769854,27679486,9,1
333308,17735086,23275624,10,1


In [21]:
pts_diag.loc[(pts_diag['subject_id']==14975184) & (pts_diag['hadm_id'] == 27635276)]

,subject_id,hadm_id,seq_num,icd_code,icd_version
2365061,14975184,27635276,1,44023,9
2365062,14975184,27635276,1,A419,10
2365063,14975184,27635276,2,5856,9
2365064,14975184,27635276,2,N186,10
2365065,14975184,27635276,3,40391,9
2365066,14975184,27635276,3,K631,10
2365067,14975184,27635276,4,25040,9
2365068,14975184,27635276,4,K559,10
2365069,14975184,27635276,5,41401,9
2365070,14975184,27635276,5,Z7682,10


In [22]:
pts_diag.groupby(['subject_id', 'hadm_id', 'icd_version'])['icd_code'].apply(list)

subject_id  hadm_id   icd_version
10000032    22595853  9              [5723, 78959, 5715, 07070, 496, 29680, 30981, ...
            22841357  9               [07071, 78959, 2875, 2761, 496, 5715, V08, 3051]
            25742920  9              [07054, 78959, V462, 5715, 2767, 2761, 496, V0...
            29079034  9              [45829, 07044, 7994, 2761, 78959, 2767, 3051, ...
10000068    25022803  9                                                        [30500]
                                                           ...                        
19999828    25744818  10             [T8141XA, E1110, K632, D682, L02211, T83728A, ...
            29734428  10             [T8131XA, T8183XA, K632, D6851, N390, Y838, Y9...
19999840    21033226  9              [3453, 51881, 5070, 5180, 42741, 43821, 43811,...
            26071774  9                 [43491, 43820, 34590, 43811, 4019, 2724, 3051]
19999987    23865745  9              [431, 3485, 20280, 5849, 5990, 2449, 41401, 78...
Name: icd

In [23]:
pts_diag.head()
adm_pts.head()
pts.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,min_year_group,max_year_group,min_age,base_anchor_year
0,10000032,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
1,10000048,F,23,2126,2008 - 2010,0,2008,2010,21,2126
2,10000068,F,19,2160,2008 - 2010,0,2008,2010,17,2160
3,10000084,M,72,2160,2017 - 2019,2161-02-13 00:00:00,2017,2019,61,2151
4,10000102,F,27,2136,2008 - 2010,0,2008,2010,25,2136


In [24]:
pts_diag.groupby(['subject_id', 'hadm_id', 'icd_version'])['icd_code'].apply(list)

subject_id  hadm_id   icd_version
10000032    22595853  9              [5723, 78959, 5715, 07070, 496, 29680, 30981, ...
            22841357  9               [07071, 78959, 2875, 2761, 496, 5715, V08, 3051]
            25742920  9              [07054, 78959, V462, 5715, 2767, 2761, 496, V0...
            29079034  9              [45829, 07044, 7994, 2761, 78959, 2767, 3051, ...
10000068    25022803  9                                                        [30500]
                                                           ...                        
19999828    25744818  10             [T8141XA, E1110, K632, D682, L02211, T83728A, ...
            29734428  10             [T8131XA, T8183XA, K632, D6851, N390, Y838, Y9...
19999840    21033226  9              [3453, 51881, 5070, 5180, 42741, 43821, 43811,...
            26071774  9                 [43491, 43820, 34590, 43811, 4019, 2724, 3051]
19999987    23865745  9              [431, 3485, 20280, 5849, 5990, 2449, 41401, 78...
Name: icd

In [25]:
# df['new_col'] = np.array(mylist)
adm_pts

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,min_year_group,max_year_group,min_age,base_anchor_year
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,...,0,F,52,2180,2014 - 2016,2180-09-09 00:00:00,2014,2016,44,2174
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,...,0,F,19,2160,2008 - 2010,0,2008,2010,17,2160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,0,F,46,2147,2017 - 2019,0,2017,2019,35,2138
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,...,0,F,46,2147,2017 - 2019,0,2017,2019,35,2138
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,...,1,M,58,2164,2008 - 2010,2164-09-17 00:00:00,2008,2010,56,2164
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,...,0,M,58,2164,2008 - 2010,2164-09-17 00:00:00,2008,2010,56,2164
